## Setup and Data Preparation

Initial setup steps to prepare the working environment and extract ClinVar data.

# ClinVar Coding Variants Data Processing

This notebook processes ClinVar coding variants data by extracting additional information including gene names, gene IDs, and associated diseases from ClinVar XML records.

## Overview

The workflow includes:
1. **Data Extraction**: Filter ClinVar entries from VEP-annotated pathogenic coding variants
2. **XML Processing**: Parse ClinVar XML records to extract gene and disease information
3. **Gene Annotation**: Map gene IDs to gene names using NCBI Entrez utilities
4. **Data Integration**: Combine all information into a comprehensive dataset

## Requirements

- Python 3.7+
- pandas library
- xml.etree.ElementTree (built-in)
- NCBI Entrez Direct tools (for gene name mapping)
- Input data: VEP-annotated pathogenic coding variants CSV file

## Data Structure

The processing creates a dataset with the following key columns:
- Variant information (chromosome, position, alleles)
- ClinVar ID and significance
- Gene symbols and IDs
- Associated disease/phenotype information

In [ ]:
# Create working directory for ClinVar data processing
import os
os.makedirs('clinvar', exist_ok=True)
print("✅ Created 'clinvar' directory")

In [ ]:
import os

# Navigate to clinvar directory
os.chdir('clinvar')
print(f"📁 Current working directory: {os.getcwd()}")

with open('vep_pathogenic_coding.csv') as infile, open('clinvar_coding_raw.csv', 'w') as outfile:
    for line in infile:
        if 'ClinVar' in line:
            outfile.write(line)

In [ ]:
# Extract ClinVar entries from VEP-annotated pathogenic coding variants
# Note: Update the input file path to match your data location
input_file = "../data/vep_pathogenic_coding.csv"  # Adjust path as needed
output_file = "clinvar_coding_raw.csv"

# Use shell command to filter ClinVar entries
import subprocess
try:
    result = subprocess.run(
        ["grep", "ClinVar", input_file],
        capture_output=True,
        text=True,
        check=True
    )
    
    with open(output_file, 'w') as f:
        f.write(result.stdout)
    
    print(f"✅ Extracted ClinVar entries to {output_file}")
    print(f"📊 Found {len(result.stdout.strip().split('\n'))} ClinVar entries")
    
except subprocess.CalledProcessError:
    print(f"❌ Error: Could not find ClinVar entries in {input_file}")
    print("Please ensure the input file exists and contains ClinVar annotations")
except FileNotFoundError:
    print(f"❌ Error: Input file {input_file} not found")
    print("Please update the input_file path to point to your VEP-annotated data")

In [ ]:
# Extract ClinVar IDs from the filtered data (assuming ID is in column 8)
# Note: Adjust column number if your data structure is different
import pandas as pd

try:
    # Read the raw ClinVar data to determine structure
    df_temp = pd.read_csv("clinvar_coding_raw.csv")
    print(f"📋 Data shape: {df_temp.shape}")
    print(f"📋 Columns: {list(df_temp.columns)}")
    
    # Extract ClinVar IDs (adjust column index as needed)
    # Column 8 corresponds to index 7 in Python (0-based)
    if df_temp.shape[1] >= 8:
        clinvar_ids = df_temp.iloc[:, 7]  # 8th column (0-based index 7)
        
        # Save IDs to file
        with open("Clinvar_ID.txt", 'w') as f:
            for id_val in clinvar_ids:
                if pd.notna(id_val):
                    f.write(f"{id_val}\n")
        
        print(f"✅ Extracted {len(clinvar_ids.dropna())} ClinVar IDs to Clinvar_ID.txt")
    else:
        print(f"❌ Error: Expected at least 8 columns, found {df_temp.shape[1]}")
        
except FileNotFoundError:
    print("❌ Error: clinvar_coding_raw.csv not found")
    print("Please run the previous cell first to extract ClinVar data")
except Exception as e:
    print(f"❌ Error processing ClinVar data: {e}")

In [ ]:
chmod +x Clinvar_esearch.sh

## XML Data Retrieval

**Note**: This step requires creating a shell script (`Clinvar_esearch.sh`) to fetch XML data from NCBI.

The script should:
1. Read ClinVar IDs from `Clinvar_ID.txt`
2. Use NCBI Entrez Direct tools to fetch XML records
3. Save XML files in a `data/` subdirectory

Example script content:
```bash
#!/bin/bash
mkdir -p data
while read -r id; do
    esearch -db clinvar -query "$id" | efetch -format xml > "data/${id}.xml"
    echo "Downloaded XML for ClinVar ID: $id"
done < Clinvar_ID.txt
```

**Prerequisites**: Install NCBI Entrez Direct tools:
- macOS: `brew install brewsci/bio/edirect`
- Linux: Follow NCBI EDirect installation guide

In [ ]:
# Parsing XML for Gene and Disease

# Make the ClinVar search script executable and run it
# Note: This assumes you have created the Clinvar_esearch.sh script

import os
import subprocess

script_path = "Clinvar_esearch.sh"

if os.path.exists(script_path):
    # Make script executable
    os.chmod(script_path, 0o755)
    print(f"✅ Made {script_path} executable")
    
    # Optionally run the script (uncomment if you want to execute automatically)
    # print("🚀 Running ClinVar XML download script...")
    # result = subprocess.run([f"./{script_path}"], capture_output=True, text=True)
    # if result.returncode == 0:
    #     print("✅ XML download completed successfully")
    # else:
    #     print(f"❌ Script execution failed: {result.stderr}")
else:
    print(f"⚠️ Warning: {script_path} not found")
    print("Please create this script manually to download ClinVar XML data")
    print("See the documentation in the previous cell for script template")

In [ ]:
# Import required libraries
import pandas as pd
import xml.etree.ElementTree as ET
import json
import os
from pathlib import Path

print("📚 Libraries imported successfully")
print(f"📁 Current directory: {os.getcwd()}")
print(f"📊 Pandas version: {pd.__version__}")

In [ ]:
# Load the raw ClinVar data
try:
    clinvar_raw = pd.read_csv("clinvar_coding_raw.csv")
    print(f"✅ Loaded ClinVar data: {clinvar_raw.shape[0]} rows, {clinvar_raw.shape[1]} columns")
    print(f"📋 Columns: {list(clinvar_raw.columns)[:10]}")  # Show first 10 columns
    
except FileNotFoundError:
    print("❌ Error: clinvar_coding_raw.csv not found")
    print("Please run the data extraction steps first")
    clinvar_raw = None
except Exception as e:
    print(f"❌ Error loading data: {e}")
    clinvar_raw = None

In [ ]:
# Remove unnecessary columns to streamline the dataset
# Note: Adjust column names based on your actual data structure

if clinvar_raw is not None:
    columns_to_remove = [
        "GENOMIC_MUTATION_ID", "N_SAMPLES", "TOTAL_SAMPLES", "FREQ", 
        "OMIM", "PMID", "AC", "AN", "AF", "MAF", "MAC"
    ]
    
    # Only remove columns that actually exist in the dataset
    existing_columns = [col for col in columns_to_remove if col in clinvar_raw.columns]
    missing_columns = [col for col in columns_to_remove if col not in clinvar_raw.columns]
    
    if existing_columns:
        clinvar_raw = clinvar_raw.drop(columns=existing_columns)
        print(f"✅ Removed {len(existing_columns)} columns: {existing_columns}")
    
    if missing_columns:
        print(f"ℹ️ Columns not found (skipped): {missing_columns}")
    
    print(f"📊 Remaining columns: {clinvar_raw.shape[1]}")
else:
    print("⚠️ Skipping column removal - data not loaded")

In [ ]:
clinvar_raw

# Preview the cleaned dataset
if clinvar_raw is not None:
    print(f"📊 Dataset shape: {clinvar_raw.shape}")
    print(f"📋 Column names: {list(clinvar_raw.columns)}")
    print("\n🔍 First few rows:")
    display(clinvar_raw.head())
    
    # Check for any null values
    null_counts = clinvar_raw.isnull().sum()
    if null_counts.sum() > 0:
        print("\n⚠️ Null values found:")
        print(null_counts[null_counts > 0])
else:
    print("❌ No data to display")

,CHROM,POS,REF,ALT,LABEL,SOURCE,CONSEQUENCE,ID,REVIEW_STATUS,GENE,split,INT_LABEL
0,chr1,976215,A,G,Pathogenic,ClinVar,missense_variant,1320032,no_assertion_criteria_provided,NaN,train,1
1,chr1,1050449,G,A,Pathogenic,ClinVar,missense_variant,1284257,no_assertion_criteria_provided,NaN,train,1
2,chr1,1050575,G,C,Pathogenic,ClinVar,missense_variant,18241,no_assertion_criteria_provided,NaN,train,1
3,chr1,1213738,G,A,Pathogenic,ClinVar,missense_variant,96692,no_assertion_criteria_provided,NaN,train,1
4,chr1,1232279,A,G,Pathogenic,ClinVar,"initiatior_codon_variant,missense_variant",60484,"criteria_provided,_multiple_submitters,_no_con...",NaN,train,1
...,...,...,...,...,...,...,...,...,...,...,...,...
22249,chrY,2787412,C,T,Pathogenic,ClinVar,missense_variant,9747,no_assertion_criteria_provided,NaN,train,1
22250,chrY,2787426,C,G,Pathogenic,ClinVar,missense_variant,9739,"criteria_provided,_single_submitter",NaN,train,1
22251,chrY,2787515,C,A,Pathogenic,ClinVar,missense_variant,492908,no_assertion_criteria_provided,NaN,train,1
22252,chrY,2787551,C,T,Pathogenic,ClinVar,missense_variant,9754,no_assertion_criteria_provided,NaN,train,1


In [ ]:
# Add new columns for gene information
if clinvar_raw is not None:
    clinvar_raw['GENE_ID'] = ""
    clinvar_raw['GENE'] = ""
    print("✅ Added GENE_ID and GENE columns")
    print(f"📊 Updated dataset shape: {clinvar_raw.shape}")
else:
    print("⚠️ Cannot add columns - data not loaded")

In [ ]:
clinvar_raw

# Display updated dataset with new columns
if clinvar_raw is not None:
    print(f"📊 Dataset with new columns: {clinvar_raw.shape}")
    print(f"📋 All columns: {list(clinvar_raw.columns)}")
    display(clinvar_raw.head())
else:
    print("❌ No data to display")

,CHROM,POS,REF,ALT,LABEL,SOURCE,CONSEQUENCE,ID,REVIEW_STATUS,GENE,split,INT_LABEL,GENE_ID
0,chr1,976215,A,G,Pathogenic,ClinVar,missense_variant,1320032,no_assertion_criteria_provided,,train,1,
1,chr1,1050449,G,A,Pathogenic,ClinVar,missense_variant,1284257,no_assertion_criteria_provided,,train,1,
2,chr1,1050575,G,C,Pathogenic,ClinVar,missense_variant,18241,no_assertion_criteria_provided,,train,1,
3,chr1,1213738,G,A,Pathogenic,ClinVar,missense_variant,96692,no_assertion_criteria_provided,,train,1,
4,chr1,1232279,A,G,Pathogenic,ClinVar,"initiatior_codon_variant,missense_variant",60484,"criteria_provided,_multiple_submitters,_no_con...",,train,1,
...,...,...,...,...,...,...,...,...,...,...,...,...,...
22249,chrY,2787412,C,T,Pathogenic,ClinVar,missense_variant,9747,no_assertion_criteria_provided,,train,1,
22250,chrY,2787426,C,G,Pathogenic,ClinVar,missense_variant,9739,"criteria_provided,_single_submitter",,train,1,
22251,chrY,2787515,C,A,Pathogenic,ClinVar,missense_variant,492908,no_assertion_criteria_provided,,train,1,
22252,chrY,2787551,C,T,Pathogenic,ClinVar,missense_variant,9754,no_assertion_criteria_provided,,train,1,


In [ ]:
import pandas as pd
import xml.etree.ElementTree as ET
import os

# Parse ClinVar XML files to extract gene information
# This processes each ClinVar ID and extracts gene symbols and IDs from XML records

if clinvar_raw is not None:
    # Load list of ClinVar IDs
    try:
        with open("Clinvar_ID.txt", "r") as f:
            clinvar_ids = [line.strip() for line in f if line.strip()]
        
        print(f"📋 Processing {len(clinvar_ids)} ClinVar IDs")
        
        processed_count = 0
        error_count = 0
        
        # Process each ClinVar ID
        for i, clinvar_id in enumerate(clinvar_ids):
            if i % 100 == 0:  # Progress indicator
                print(f"📊 Processing ID {i+1}/{len(clinvar_ids)}...")
            
            try:
                id_int = int(clinvar_id)
                xml_path = f'data/{clinvar_id}.xml'
                
                # Check if XML file exists
                if not os.path.exists(xml_path):
                    print(f"⚠️ XML file not found: {xml_path}")
                    continue
                
                # Parse XML file
                with open(xml_path, 'r', encoding='utf-8') as file:
                    tree = ET.parse(file)
                    root = tree.getroot()
                
                # Check for error in XML
                error_element = root.find(".//error")
                if error_element is not None:
                    # Remove entries with errors
                    clinvar_raw = clinvar_raw[clinvar_raw["ID"] != id_int]
                    error_count += 1
                    continue
                
                # Extract gene information
                gene_names = []
                gene_ids = []
                
                for gene in root.findall(".//genes/gene"):
                    symbol = gene.findtext("symbol")
                    gene_id_data = gene.findtext("GeneID")
                    
                    if symbol:
                        gene_names.append(symbol)
                    if gene_id_data:
                        gene_ids.append(gene_id_data)
                
                # Join multiple entries with commas
                gene_name_str = ", ".join(gene_names) if gene_names else ""
                gene_id_str = ", ".join(gene_ids) if gene_ids else ""
                
                # Update DataFrame
                mask = clinvar_raw["ID"] == id_int
                if mask.any():
                    clinvar_raw.loc[mask, "GENE"] = gene_name_str
                    clinvar_raw.loc[mask, "GENE_ID"] = gene_id_str
                    processed_count += 1
                
            except ET.ParseError as e:
                print(f"⚠️ XML parsing error for {clinvar_id}: {e}")
                error_count += 1
            except ValueError as e:
                print(f"⚠️ Invalid ClinVar ID {clinvar_id}: {e}")
                error_count += 1
            except Exception as e:
                print(f"⚠️ Unexpected error processing {clinvar_id}: {e}")
                error_count += 1
        
        print(f"\n✅ Processing complete:")
        print(f"   📊 Successfully processed: {processed_count}")
        print(f"   ❌ Errors encountered: {error_count}")
        print(f"   📋 Final dataset shape: {clinvar_raw.shape}")
        
    except FileNotFoundError:
        print("❌ Error: Clinvar_ID.txt not found")
        print("Please run the ID extraction step first")
    except Exception as e:
        print(f"❌ Error during XML processing: {e}")
else:
    print("⚠️ Cannot process XML files - ClinVar data not loaded")

In [ ]:
clinvar_raw

# Display the dataset with extracted gene information
if clinvar_raw is not None:
    print(f"📊 Dataset after gene extraction: {clinvar_raw.shape}")
    
    # Show statistics
    gene_filled = (clinvar_raw['GENE'] != '').sum()
    gene_id_filled = (clinvar_raw['GENE_ID'] != '').sum()
    
    print(f"📋 Entries with gene names: {gene_filled} ({gene_filled/len(clinvar_raw)*100:.1f}%)")
    print(f"📋 Entries with gene IDs: {gene_id_filled} ({gene_id_filled/len(clinvar_raw)*100:.1f}%)")
    
    # Show sample data
    display(clinvar_raw.head(10))
else:
    print("❌ No data to display")

,CHROM,POS,REF,ALT,LABEL,SOURCE,CONSEQUENCE,ID,REVIEW_STATUS,GENE,split,INT_LABEL,GENE_ID
0,chr1,976215,A,G,Pathogenic,ClinVar,missense_variant,1320032,no_assertion_criteria_provided,PERM1,train,1,84808
1,chr1,1050449,G,A,Pathogenic,ClinVar,missense_variant,1284257,no_assertion_criteria_provided,AGRN,train,1,375790
2,chr1,1050575,G,C,Pathogenic,ClinVar,missense_variant,18241,no_assertion_criteria_provided,AGRN,train,1,375790
3,chr1,1213738,G,A,Pathogenic,ClinVar,missense_variant,96692,no_assertion_criteria_provided,TNFRSF4,train,1,7293
4,chr1,1232279,A,G,Pathogenic,ClinVar,"initiatior_codon_variant,missense_variant",60484,"criteria_provided,_multiple_submitters,_no_con...",B3GALT6,train,1,126792
...,...,...,...,...,...,...,...,...,...,...,...,...,...
22249,chrY,2787412,C,T,Pathogenic,ClinVar,missense_variant,9747,no_assertion_criteria_provided,SRY,train,1,6736
22250,chrY,2787426,C,G,Pathogenic,ClinVar,missense_variant,9739,"criteria_provided,_single_submitter",SRY,train,1,6736
22251,chrY,2787515,C,A,Pathogenic,ClinVar,missense_variant,492908,no_assertion_criteria_provided,SRY,train,1,6736
22252,chrY,2787551,C,T,Pathogenic,ClinVar,missense_variant,9754,no_assertion_criteria_provided,SRY,train,1,6736


## Disease/Phenotype Information Extraction

This section extracts disease and phenotype information from the ClinVar XML records. Each variant may be associated with multiple diseases, so the data is expanded to create one row per variant-disease combination.

### Putting in the Disease Name

In [ ]:
# Extract disease/phenotype information from ClinVar XML files
# This creates multiple rows for variants associated with multiple diseases

if clinvar_raw is not None:
    try:
        # Load ClinVar IDs
        with open("Clinvar_ID.txt", "r") as f:
            clinvar_ids = [line.strip() for line in f if line.strip()]
        
        print(f"📋 Processing {len(clinvar_ids)} ClinVar IDs for disease extraction")
        
        # Ensure ID column is integer type
        clinvar_raw["ID"] = clinvar_raw["ID"].astype(int)
        
        # Create new DataFrame to store expanded data
        clinvar_data = pd.DataFrame(columns=clinvar_raw.columns.tolist() + ["Disease"])
        
        processed_count = 0
        disease_count = 0
        
        # Process each ClinVar ID
        for i, clinvar_id in enumerate(clinvar_ids):
            if i % 100 == 0:  # Progress indicator
                print(f"📊 Processing disease info {i+1}/{len(clinvar_ids)}...")
            
            try:
                id_int = int(clinvar_id)
                xml_path = f"data/{clinvar_id}.xml"
                
                if not os.path.exists(xml_path):
                    continue
                
                # Parse XML
                tree = ET.parse(xml_path)
                root = tree.getroot()
                
                # Extract all trait names (diseases/phenotypes)
                trait_names = []
                for trait in root.findall(".//trait"):
                    trait_name = trait.findtext("trait_name")
                    if trait_name:
                        trait_names.append(trait_name)
                
                # Filter out 'not provided' if other traits exist
                filtered_traits = [t for t in trait_names if t.lower() != "not provided"]
                if not filtered_traits and "not provided" in [t.lower() for t in trait_names]:
                    filtered_traits = ["not provided"]
                
                # If no traits found, use empty string
                if not filtered_traits:
                    filtered_traits = [""]
                
                # Create one row for each disease/trait
                base_row = clinvar_raw[clinvar_raw["ID"] == id_int]
                if not base_row.empty:
                    for disease_name in filtered_traits:
                        new_row = base_row.copy()
                        new_row["Disease"] = disease_name
                        clinvar_data = pd.concat([clinvar_data, new_row], ignore_index=True)
                        disease_count += 1
                    processed_count += 1
                
            except ET.ParseError as e:
                print(f"⚠️ XML parsing error for {clinvar_id}: {e}")
            except Exception as e:
                print(f"⚠️ Error processing {clinvar_id}: {e}")
        
        print(f"\n✅ Disease extraction complete:")
        print(f"   📊 Variants processed: {processed_count}")
        print(f"   🔬 Total variant-disease pairs: {disease_count}")
        print(f"   📋 Final dataset shape: {clinvar_data.shape}")
        
        # Save intermediate results
        clinvar_data.to_csv("clinvar_with_disease.csv", sep='\t', index=False)
        print("💾 Saved results to clinvar_with_disease.csv")
        
    except FileNotFoundError:
        print("❌ Error: Required files not found")
        print("Please ensure Clinvar_ID.txt exists and XML files are downloaded")
        clinvar_data = None
    except Exception as e:
        print(f"❌ Error during disease extraction: {e}")
        clinvar_data = None
else:
    print("⚠️ Cannot extract diseases - ClinVar data not loaded")
    clinvar_data = None

In [ ]:
clinvar_data

# Display the dataset with disease information
if 'clinvar_data' in locals() and clinvar_data is not None:
    print(f"📊 Dataset with diseases: {clinvar_data.shape}")
    
    # Show disease statistics
    disease_counts = clinvar_data['Disease'].value_counts()
    print(f"\n🔬 Disease distribution (top 10):")
    print(disease_counts.head(10))
    
    # Show sample data
    print("\n🔍 Sample data:")
    display(clinvar_data.head())
else:
    print("❌ No disease data to display")

,CHROM,POS,REF,ALT,LABEL,SOURCE,CONSEQUENCE,ID,REVIEW_STATUS,GENE,split,INT_LABEL,GENE_ID,Disease
0,chr1,976215,A,G,Pathogenic,ClinVar,missense_variant,1320032,no_assertion_criteria_provided,PERM1,train,1,84808,Renal tubular epithelial cell apoptosis
1,chr1,976215,A,G,Pathogenic,ClinVar,missense_variant,1320032,no_assertion_criteria_provided,PERM1,train,1,84808,Neutrophil inclusion bodies
2,chr1,1050449,G,A,Pathogenic,ClinVar,missense_variant,1284257,no_assertion_criteria_provided,AGRN,train,1,375790,Congenital myasthenic syndrome 8
3,chr1,1050575,G,C,Pathogenic,ClinVar,missense_variant,18241,no_assertion_criteria_provided,AGRN,train,1,375790,Congenital myasthenic syndrome 8
4,chr1,1213738,G,A,Pathogenic,ClinVar,missense_variant,96692,no_assertion_criteria_provided,TNFRSF4,train,1,7293,Combined immunodeficiency due to OX40 deficiency
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32680,chrY,2787412,C,T,Pathogenic,ClinVar,missense_variant,9747,no_assertion_criteria_provided,SRY,train,1,6736,"46,XY sex reversal 1"
32681,chrY,2787426,C,G,Pathogenic,ClinVar,missense_variant,9739,"criteria_provided,_single_submitter",SRY,train,1,6736,not provided
32682,chrY,2787515,C,A,Pathogenic,ClinVar,missense_variant,492908,no_assertion_criteria_provided,SRY,train,1,6736,"46,XY sex reversal 1"
32683,chrY,2787551,C,T,Pathogenic,ClinVar,missense_variant,9754,no_assertion_criteria_provided,SRY,train,1,6736,"46,XY sex reversal 1"


In [ ]:
# Count entries with 'not provided' disease information
if 'clinvar_data' in locals() and clinvar_data is not None:
    not_provided_count = (clinvar_data["Disease"] == "not provided").sum()
    total_count = len(clinvar_data)
    
    print(f"📊 Entries with 'not provided' disease: {not_provided_count}")
    print(f"📊 Total entries: {total_count}")
    print(f"📊 Percentage: {not_provided_count/total_count*100:.1f}%")
else:
    print("❌ Cannot calculate statistics - data not available")

np.int64(2749)

## Gene ID to Gene Name Mapping

This section converts gene IDs to human-readable gene names using NCBI Entrez utilities.

**Prerequisites**: NCBI Entrez Direct tools must be installed:
- macOS: `brew install brewsci/bio/edirect`
- Linux: Follow NCBI EDirect installation guide

The process:
1. Extract unique gene IDs from the dataset
2. Use `esummary` to fetch gene descriptions from NCBI
3. Create a mapping dictionary
4. Apply the mapping to add gene names to the dataset

In [ ]:
#!/usr/bin/env python3

import os
import pandas as pd

# Extract unique gene IDs and create mapping file
# This prepares the gene ID list for NCBI lookup

if 'clinvar_data' in locals() and clinvar_data is not None:
    # Extract all unique gene IDs
    all_gene_ids = set()
    
    for gene_id_str in clinvar_data['GENE_ID'].dropna():
        if gene_id_str.strip():  # Skip empty strings
            # Split comma-separated IDs
            ids = [gid.strip() for gid in gene_id_str.split(',') if gid.strip()]
            all_gene_ids.update(ids)
    
    # Save unique gene IDs to file
    with open("gene_id.txt", 'w') as f:
        for gene_id in sorted(all_gene_ids):
            f.write(f"{gene_id}\n")
    
    print(f"✅ Extracted {len(all_gene_ids)} unique gene IDs to gene_id.txt")
    
    # Create the shell script for NCBI lookup
    script_content = '''#!/bin/bash

input_file="gene_id.txt"
output_file="gene_id_to_name.json"

# Check if input file exists
if [ ! -f "$input_file" ]; then
    echo "❌ Error: $input_file not found"
    exit 1
fi

# Check if EDirect tools are available
if ! command -v esummary &> /dev/null; then
    echo "❌ Error: NCBI EDirect tools not found"
    echo "Please install: brew install brewsci/bio/edirect (macOS)"
    exit 1
fi

echo "🚀 Starting gene ID to name mapping..."

# Start JSON object
echo "{" > "$output_file"

first_entry=true
total_lines=$(wc -l < "$input_file")
current_line=0

while IFS= read -r gene_id; do
    # Skip empty lines
    [[ -z "$gene_id" ]] && continue
    
    current_line=$((current_line + 1))
    
    # Progress indicator
    if (( current_line % 50 == 0 )); then
        echo "📊 Processing $current_line/$total_lines gene IDs..."
    fi
    
    # Fetch gene description using Entrez Direct
    description=$(esummary -db gene -id "$gene_id" 2>/dev/null | xtract -pattern DocumentSummary -element Description)
    
    # Handle empty description
    if [ -z "$description" ]; then
        description="Unknown"
    fi
    
    # JSON escape quotes and other special characters
    description=$(printf '%s' "$description" | sed 's/"/\\"/g')
    
    # Add comma if not the first entry
    if [ "$first_entry" = true ]; then
        first_entry=false
    else
        echo "," >> "$output_file"
    fi
    
    # Append key-value pair
    echo "  \"$gene_id\": \"$description\"" >> "$output_file"
    
done < "$input_file"

# Close JSON object
echo "" >> "$output_file"
echo "}" >> "$output_file"

echo "✅ Gene ID to name mapping completed"
echo "💾 Results saved to $output_file"
'''
    
    # Write the script
    with open("gene_mapping.sh", 'w') as f:
        f.write(script_content)
    
    # Make executable
    os.chmod("gene_mapping.sh", 0o755)
    
    print("✅ Created gene_mapping.sh script")
    print("\n🚀 To run the gene mapping:")
    print("   ./gene_mapping.sh")
    print("\n⚠️ Note: This requires NCBI EDirect tools to be installed")
    
else:
    print("⚠️ Cannot create gene mapping - data not available")

## Apply Gene Name Mapping

Load the gene ID to name mapping and apply it to the dataset to add human-readable gene names.

Read json and add it to the clinvar_data df

In [ ]:
import json

# Load gene ID to name mapping and apply to dataset

if 'clinvar_data' in locals() and clinvar_data is not None:
    try:
        # Load gene ID → name mapping
        with open("gene_id_to_name.json", "r") as f:
            gene_id_dict = json.load(f)
        
        print(f"✅ Loaded mapping for {len(gene_id_dict)} gene IDs")
        
        # Function to convert gene IDs to gene names
        def get_gene_names(gene_id_str):
            if pd.isna(gene_id_str) or not gene_id_str.strip():
                return ""
            
            gene_ids = [gid.strip() for gid in gene_id_str.split(",") if gid.strip()]
            gene_names = []
            
            for gid in gene_ids:
                gene_name = gene_id_dict.get(gid, f"Unknown_ID_{gid}")
                gene_names.append(gene_name)
            
            return " | ".join(gene_names)
        
        # Apply mapping to create gene names column
        print("📊 Applying gene name mapping...")
        clinvar_data["GENE_Name"] = clinvar_data["GENE_ID"].apply(get_gene_names)
        
        # Statistics
        mapped_count = (clinvar_data["GENE_Name"] != "").sum()
        print(f"✅ Gene names mapped for {mapped_count} entries ({mapped_count/len(clinvar_data)*100:.1f}%)")
        
        # Show sample mappings
        sample_data = clinvar_data[clinvar_data["GENE_Name"] != ""][["GENE_ID", "GENE_Name"]].head()
        if not sample_data.empty:
            print("\n🔍 Sample gene ID to name mappings:")
            for _, row in sample_data.iterrows():
                print(f"   {row['GENE_ID']} → {row['GENE_Name'][:100]}{'...' if len(row['GENE_Name']) > 100 else ''}")
        
    except FileNotFoundError:
        print("❌ Error: gene_id_to_name.json not found")
        print("Please run the gene mapping script first: ./gene_mapping.sh")
        # Create empty column as fallback
        clinvar_data["GENE_Name"] = ""
    except json.JSONDecodeError as e:
        print(f"❌ Error parsing JSON mapping file: {e}")
        clinvar_data["GENE_Name"] = ""
    except Exception as e:
        print(f"❌ Error applying gene mapping: {e}")
        clinvar_data["GENE_Name"] = ""
else:
    print("⚠️ Cannot apply gene mapping - data not available")

In [ ]:
# Display final dataset with all extracted information
if 'clinvar_data' in locals() and clinvar_data is not None:
    print(f"📊 Final dataset shape: {clinvar_data.shape}")
    print(f"📋 Columns: {list(clinvar_data.columns)}")
    
    # Data completeness statistics
    print("\n📈 Data Completeness:")
    for col in ['GENE', 'GENE_ID', 'GENE_Name', 'Disease']:
        if col in clinvar_data.columns:
            filled_count = (clinvar_data[col] != '').sum()
            print(f"   {col}: {filled_count}/{len(clinvar_data)} ({filled_count/len(clinvar_data)*100:.1f}%)")
    
    # Sample data
    print("\n🔍 Sample data:")
    display(clinvar_data.head())
    
    # Memory usage
    memory_mb = clinvar_data.memory_usage(deep=True).sum() / 1024 / 1024
    print(f"\n💾 Dataset memory usage: {memory_mb:.1f} MB")
else:
    print("❌ No final data to display")

,CHROM,POS,REF,ALT,LABEL,SOURCE,CONSEQUENCE,ID,REVIEW_STATUS,GENE,split,INT_LABEL,GENE_ID,Disease,GENE_Name
0,chr1,976215,A,G,Pathogenic,ClinVar,missense_variant,1320032,no_assertion_criteria_provided,PERM1,train,1,84808,Renal tubular epithelial cell apoptosis,"PPARGC1 and ESRR induced regulator, muscle 1"
1,chr1,976215,A,G,Pathogenic,ClinVar,missense_variant,1320032,no_assertion_criteria_provided,PERM1,train,1,84808,Neutrophil inclusion bodies,"PPARGC1 and ESRR induced regulator, muscle 1"
2,chr1,1050449,G,A,Pathogenic,ClinVar,missense_variant,1284257,no_assertion_criteria_provided,AGRN,train,1,375790,Congenital myasthenic syndrome 8,agrin
3,chr1,1050575,G,C,Pathogenic,ClinVar,missense_variant,18241,no_assertion_criteria_provided,AGRN,train,1,375790,Congenital myasthenic syndrome 8,agrin
4,chr1,1213738,G,A,Pathogenic,ClinVar,missense_variant,96692,no_assertion_criteria_provided,TNFRSF4,train,1,7293,Combined immunodeficiency due to OX40 deficiency,TNF receptor superfamily member 4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32680,chrY,2787412,C,T,Pathogenic,ClinVar,missense_variant,9747,no_assertion_criteria_provided,SRY,train,1,6736,"46,XY sex reversal 1",sex determining region Y
32681,chrY,2787426,C,G,Pathogenic,ClinVar,missense_variant,9739,"criteria_provided,_single_submitter",SRY,train,1,6736,not provided,sex determining region Y
32682,chrY,2787515,C,A,Pathogenic,ClinVar,missense_variant,492908,no_assertion_criteria_provided,SRY,train,1,6736,"46,XY sex reversal 1",sex determining region Y
32683,chrY,2787551,C,T,Pathogenic,ClinVar,missense_variant,9754,no_assertion_criteria_provided,SRY,train,1,6736,"46,XY sex reversal 1",sex determining region Y


In [ ]:
# Show example of gene name mapping
if 'clinvar_data' in locals() and clinvar_data is not None and len(clinvar_data) > 32684:
    example_gene_name = clinvar_data.iloc[32684]['GENE_Name']
    example_gene_id = clinvar_data.iloc[32684]['GENE_ID']
    
    print(f"🔍 Example gene mapping for row 32684:")
    print(f"   Gene ID: {example_gene_id}")
    print(f"   Gene Name: {example_gene_name}")
else:
    # Show any available example
    if 'clinvar_data' in locals() and clinvar_data is not None and not clinvar_data.empty:
        # Find first row with gene name data
        example_row = clinvar_data[clinvar_data['GENE_Name'] != ''].iloc[0] if (clinvar_data['GENE_Name'] != '').any() else clinvar_data.iloc[0]
        
        print(f"🔍 Example gene mapping:")
        print(f"   Gene ID: {example_row.get('GENE_ID', 'N/A')}")
        print(f"   Gene Name: {example_row.get('GENE_Name', 'N/A')}")
    else:
        print("❌ No data available for example")

'P300/CBP strongly-dependent group 1 enhancer GRCh37_chrY:6931456-6932655| transducin beta like 1 Y-linked'

In [ ]:
import os

# Save the final processed dataset
if 'clinvar_data' in locals() and clinvar_data is not None:
    output_file = "clinvar_with_disease.csv"
    
    try:
        clinvar_data.to_csv(output_file, index=False)
        
        print(f"✅ Final dataset saved to {output_file}")
        print(f"📊 Saved {len(clinvar_data)} records with {len(clinvar_data.columns)} columns")
        
        # File size
        file_size = os.path.getsize(output_file) / 1024 / 1024
        print(f"💾 File size: {file_size:.1f} MB")
        
        # Summary of what was accomplished
        print("\n🎯 Processing Summary:")
        print(f"   ✓ Extracted ClinVar coding variants")
        print(f"   ✓ Parsed XML records for gene information")
        print(f"   ✓ Mapped diseases/phenotypes")
        print(f"   ✓ Added human-readable gene names")
        print(f"   ✓ Created comprehensive dataset")
        
    except Exception as e:
        print(f"❌ Error saving dataset: {e}")
else:
    print("⚠️ No data available to save")

In [ ]:
import os
import shutil

# Optional: Clean up temporary XML data directory
# Uncomment the following lines if you want to remove the XML files to save space

if os.path.exists("data") and os.path.isdir("data"):
    # Count files before cleanup
    xml_files = [f for f in os.listdir("data") if f.endswith('.xml')]
    
    print(f"🗂️ Found {len(xml_files)} XML files in data directory")
    
    # Uncomment to actually remove the directory
    # shutil.rmtree("data")
    # print("🗑️ Removed temporary XML data directory")
    
    print("ℹ️ XML files preserved. Uncomment the cleanup code to remove them.")
else:
    print("ℹ️ No XML data directory found to clean up")

In [ ]:
## Processing Complete ✅

The ClinVar coding variants have been successfully processed with the following enhancements:

### Generated Files:
- `clinvar_coding_raw.csv` - Raw ClinVar entries extracted from VEP data
- `Clinvar_ID.txt` - List of ClinVar IDs for processing
- `gene_id.txt` - Unique gene IDs for name mapping
- `gene_id_to_name.json` - Gene ID to name mapping dictionary
- `clinvar_with_disease.csv` - **Final comprehensive dataset**

### Dataset Features:
- **Variant Information**: Genomic coordinates, alleles, and annotations
- **Gene Data**: Symbols, IDs, and human-readable names
- **Disease/Phenotype**: Associated conditions and clinical significance
- **Expanded Format**: One row per variant-disease combination

### Next Steps:
1. **Quality Control**: Review the data for completeness and accuracy
2. **Analysis**: Use the dataset for downstream genetic analysis
3. **Integration**: Combine with other datasets as needed
4. **Documentation**: Update metadata and create data dictionary

### File Cleanup:
- XML files in `data/` directory can be removed to save space
- Intermediate files can be archived or removed as needed